## **Notebook 03: Exploratory Data Analysis & Visualization**

**Objective**: Explore analyzed data and prototype key visualizations for dashboard

**This one's all about storytelling:**
- So we have collected our data, got GPT to go through all of it and give us some interesting results
- Now its time to actually derive a meaningful narrative from everything we have.
- Firstly these visualisations have to actually be useful. I only have articles from the past few days, so trying to analyse trends over time is not a feasible option. 
- I want conciseness = We want to fit a digestible amount of information into just 4-5 visualisations
- Of course these visualisations have to be nice on the eye. This involves good graphical and thematic planning. 
- Finally they need to each tell a different story, one that can be interpreted by anyone seeing them. 

**Planned Visualizations:**
1. Market Overview - Article distribution and average relevance
2. Relevance Distribution - Box plots by market
3. Timeline Analysis - Activity over time
4. Category Breakdown - Regulatory focus areas
5. Regulator Network - Co-occurrence graph (if viable)

**Workflow:**
1. Load processed data
2. Data feasibility checks for each visualization
3. Prototype visualizations
4. Refine for dashboard implementation

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from collections import Counter
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [2]:
processed_dir = Path('../data/processed')
latest_file = sorted(processed_dir.glob('analyzed_articles_*.json'))[-1]

print(f"\nLoading: {latest_file.name}")
df = pd.read_json(latest_file)
df['publishedAt'] = pd.to_datetime(df['publishedAt'])

print(f"Loaded: {len(df)} articles")
print(f"Columns: {len(df.columns)}")
print(f"\nFirst check - data types:")
print(df.dtypes)


Loading: analyzed_articles_20251102_220304.json
Loaded: 652 articles
Columns: 19

First check - data types:
author                          object
title                           object
description                     object
url                             object
urlToImage                      object
publishedAt        datetime64[ns, UTC]
content                         object
source_type                     object
market_id                       object
market_name                     object
source_name                     object
article_index                    int64
relevance_score                  int64
impact_level                    object
category                        object
key_regulators                  object
what_happened                   object
why_relevant                    object
confidence                      object
dtype: object


### **Exploratory Data Analysis**

- What Im Doing: Test data coverage for each planned visualization
- Why: Identify which visualizations are viable before building them
- Technical Note: Statistical checks, data completeness analysis

In [10]:

all_regulators = []
for regs in df['key_regulators']:
    if isinstance(regs, list):
        all_regulators.extend(regs)
unique_regulators = len(set(all_regulators))
total_mentions = len(all_regulators)
print(f"Unique regulators: {unique_regulators}")
print(f"Total mentions: {total_mentions}")
print(f"Avg per article: {total_mentions/len(df):.1f}")
if unique_regulators > 0:
    top_regs = Counter(all_regulators).most_common(10)
    print(f"Top 10 regulators:")
    for reg, count in top_regs:
        print(f"  - {reg}: {count}")
print(f"Status: {'VIABLE' if unique_regulators > 15 else 'LIMITED - consider alternative'}")

Unique regulators: 171
Total mentions: 704
Avg per article: 1.1
Top 10 regulators:
  - Bank of Japan: 83
  - Federal Reserve: 52
  - SEC: 47
  - Bank of England: 40
  - Financial Conduct Authority (FCA): 40
  - European Central Bank: 34
  - CFTC: 23
  - Bundesbank: 21
  - Banque de France: 19
  - FCA: 17
Status: VIABLE


In [8]:
# Checking out data on different categories
print(df['category'].value_counts())
print(f"Unique categories: {df['category'].nunique()}")
print("Status: VIABLE")

# 4. Market comparison
print("\n4. MARKET OVERVIEW:")
market_stats = df.groupby('market_name').agg({
    'relevance_score': ['count', 'mean', 'std']
}).round(2)
print(market_stats)
print("Status: VIABLE")

category
securities_regulation      196
monetary_policy            155
banking_regulation          91
other                       88
central_banking             60
market_infrastructure       50
cross_border_investment     12
Name: count, dtype: int64
Unique categories: 7
Status: VIABLE

4. MARKET OVERVIEW:
               relevance_score            
                         count  mean   std
market_name                               
France                     129  6.09  2.51
Germany                     99  6.36  2.07
Japan                      142  6.37  2.16
United Kingdom             122  5.52  2.87
United States              160  3.74  3.44
Status: VIABLE


In [5]:
print(df['relevance_score'].describe())
print(f"Unique scores: {df['relevance_score'].nunique()}")
print("Status: VIABLE")

count    652.000000
mean       5.509202
std        2.902127
min        0.000000
25%        2.000000
50%        7.000000
75%        8.000000
max       10.000000
Name: relevance_score, dtype: float64
Unique scores: 10
Status: VIABLE


In [9]:
# Confirming that a timelimne not being feasible
valid_dates = df['publishedAt'].notna().sum()
print(f"Articles with dates: {valid_dates}/{len(df)} ({valid_dates/len(df)*100:.1f}%)")
if valid_dates > 0:
    print(f"Date range: {df['publishedAt'].min()} to {df['publishedAt'].max()}")
    print(f"Days covered: {(df['publishedAt'].max() - df['publishedAt'].min()).days}")
print(f"Status: {'VIABLE' if valid_dates > 400 else 'MARGINAL - significant missing dates'}")

Articles with dates: 381/652 (58.4%)
Date range: 2025-10-03 01:04:26+00:00 to 2025-11-01 14:13:42+00:00
Days covered: 29
Status: MARGINAL - significant missing dates


### **Colour Scheme**

- Before I jump into visualisations, I think a prudent thing to do would be to define a colour scheme. 
- This would keep my project thematically conistent, while allowing me to also really customise it exactly the way I want. 
- After a little bit of experimenting, I landed on a colour scheme that would fit well for this project

In [12]:
# Color scheme
COLORS = {
    'primary': '#1A3A52',      # Deep navy
    'accent': '#2C8C99',       # Teal
    'highlight': '#F4A261',    # Gold
    'alert': '#E76F51',        # Coral
    'secondary': '#5D6D7E',    # Slate
    'background': '#F8F9FA',   # Light gray
    
    'markets': {
        'United States': '#2C8C99',
        'Japan': '#E76F51',
        'United Kingdom': '#F4A261',
        'Germany': '#5D6D7E',
        'France': '#9B59B6'
    },
    
    'impact': {
        'high': '#E76F51',
        'medium': '#F4A261',
        'low': '#5D6D7E'
    }
}

# Plotly template
template = {
    'layout': {
        'font': {'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'},
        'plot_bgcolor': 'white',
        'paper_bgcolor': 'white',
        'title': {'font': {'size': 18, 'color': COLORS['primary']}},
        'xaxis': {'showgrid': False, 'showline': True, 'linecolor': '#E0E0E0'},
        'yaxis': {'showgrid': True, 'gridcolor': '#F0F0F0', 'showline': False}
    }
}


print("\nMarket colors:")
for market, color in COLORS['markets'].items():
    print(f"  {market}: {color}")


Market colors:
  United States: #2C8C99
  Japan: #E76F51
  United Kingdom: #F4A261
  Germany: #5D6D7E
  France: #9B59B6


### **Market Coverage and Relevance**

**What I'm Doing:**
- Aggregating article counts by market and calculating average relevance scores
- Using dual-axis visualization to show both volume (bars) and quality (line) metrics simultaneously
- Sorting markets by article count to highlight portfolio coverage priorities

**What It Shows:**
- Total regulatory news volume for each market compared to data quality as measured by relevance scores
- Whether high article volume correlates with high-relevance regulatory developments
- Which markets generate the most actionable regulatory intelligence for institutional investors

In [27]:

# Prepare data
market_overview = df.groupby('market_name').agg({
    'relevance_score': ['count', 'mean']
}).reset_index()
market_overview.columns = ['market', 'article_count', 'avg_relevance']
market_overview = market_overview.sort_values('article_count', ascending=False)

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Bar chart for article count
fig.add_trace(
    go.Bar(
        x=market_overview['market'],
        y=market_overview['article_count'],
        name='Article Count',
        marker_color=[COLORS['markets'][m] for m in market_overview['market']],
        text=market_overview['article_count'],
        textposition='outside',
        opacity=0.8
    ),
    secondary_y=False
)

# Line chart for average relevance
fig.add_trace(
    go.Scatter(
        x=market_overview['market'],
        y=market_overview['avg_relevance'],
        name='Avg Relevance',
        mode='lines+markers',
        line=dict(color=COLORS['alert'], width=3),
        marker=dict(size=10, color=COLORS['alert'], line=dict(width=2, color='white')),
        yaxis='y2'
    ),
    secondary_y=True
)

# Update layout
fig.update_layout(
    title='Regulatory News Coverage by Market',
    xaxis_title='Market',
    yaxis_title='Number of Articles',
    yaxis2_title='Average Relevance Score',
    height=500,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    hovermode='x unified',
    font=dict(family='Arial, sans-serif', size=12, color='#2C3E50'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font=dict(size=18, color=COLORS['primary']),
    xaxis=dict(showgrid=False, showline=True, linecolor='#E0E0E0'),
    yaxis=dict(showgrid=True, gridcolor='#F0F0F0', showline=False)
)

fig.update_yaxes(range=[0, market_overview['article_count'].max() * 1.25], secondary_y=False)
fig.update_yaxes(range=[0, 10], secondary_y=True)

fig.show()
print("\nMarket Overview complete")


Market Overview complete


### **Relevance Box Plot By Market**

**What I'm Doing:**

- Creating box plots to display the full distribution of relevance scores for each market
- Calculating median, quartiles, and outliers to understand score variability
- Ordering markets by median relevance score to identify highest-quality news sources

**What It Shows:**

- Consistency of regulatory relevance across different markets
- Which markets have the most consistently high-relevance articles versus sporadic high-value content
- Data quality patterns that indicate whether certain markets require different filtering thresholds

In [28]:


# Prepare data sorted by median relevance
market_order = df.groupby('market_name')['relevance_score'].median().sort_values(ascending=False).index

fig = go.Figure()

for market in market_order:
    market_data = df[df['market_name'] == market]['relevance_score']
    
    fig.add_trace(go.Box(
        y=market_data,
        name=market,
        marker_color=COLORS['markets'][market],
        boxmean='sd',
        opacity=0.8
    ))

fig.update_layout(
    title='Relevance Score Distribution by Market',
    yaxis_title='Relevance Score (0-10)',
    xaxis_title='Market',
    height=500,
    showlegend=False,
    hovermode='closest',
    font=dict(family='Arial, sans-serif', size=12, color='#2C3E50'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font=dict(size=18, color=COLORS['primary']),
    xaxis=dict(showgrid=False, showline=True, linecolor='#E0E0E0'),
    yaxis=dict(showgrid=True, gridcolor='#F0F0F0', showline=False, range=[0, 10])
)

fig.show()
print("\nRelevance Distribution complete")


Relevance Distribution complete


### **Regulatory Categories by Market**

**What I'm Doing:**

- Cross-tabulating regulatory categories against markets to show distribution patterns
- Creating grouped bar charts to enable direct comparison of category prevalence across jurisdictions
- Color-coding categories to distinguish between different types of regulatory focus

**What It Shows:**

- Which types of regulations dominate in each market
- Whether certain markets focus more heavily on specific regulatory areas
- Regulatory priorities and focus areas that may impact cross-border investment strategies

In [32]:
print("VIZ 3: CATEGORY DISTRIBUTION BY MARKET\n")

# Prepare data - cross-tabulation
category_market = pd.crosstab(df['market_name'], df['category'])

# Sort markets by total articles
market_order = category_market.sum(axis=1).sort_values(ascending=False).index
category_market = category_market.loc[market_order]

if 'cross_border_investment' in category_market.columns:
    category_market['other'] = category_market['other'] + category_market['cross_border_investment']
    category_market = category_market.drop(columns=['cross_border_investment'])

# Create figure
fig = go.Figure()

# Define category colors
category_colors = {
    'securities_regulation': '#2C8C99',
    'monetary_policy': '#F4A261', 
    'banking_regulation': '#E76F51',
    'central_banking': '#9B59B6',
    'market_infrastructure': '#5D6D7E',
    'cross_border_investment': '#1A3A52',
    'other': '#BDC3C7'
}

for category in category_market.columns:
    fig.add_trace(go.Bar(
        name=category.replace('_', ' ').title(),
        x=category_market.index,
        y=category_market[category],
        marker_color=category_colors.get(category, '#95A5A6'),
        opacity=0.85
    ))


print("\nCategory Distribution complete")

fig.update_layout(
    title='Regulatory Categories by Market',
    xaxis_title='Market',
    yaxis_title='Number of Articles',
    barmode='group',
    height=500,
    width=1200,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=1.01,
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='#E0E0E0',
        borderwidth=1
    ),
    hovermode='x unified',
    font=dict(family='Arial, sans-serif', size=12, color='#2C3E50'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font=dict(size=18, color=COLORS['primary']),
    xaxis=dict(showgrid=False, showline=True, linecolor='#E0E0E0'),
    yaxis=dict(showgrid=True, gridcolor='#F0F0F0', showline=False)
)

fig.show()

VIZ 3: CATEGORY DISTRIBUTION BY MARKET


Category Distribution complete


### **Network Mapping** 

- I tried to make a map of regulators and how often they are mentioned together
- The problem is thatmultiple names for the same regulator are represented different (eg federal reserve and fed). This CAN be fixed, but I do not think thisgraph tells a compelling enough story
- Another problem is that we are covering multiple markets, each with different regulators. 
- Therefore I eventually decided tonotinclude it in the final dashboard.

In [36]:
import networkx as nx
from itertools import combinations

# Build co-occurrence network
edges = []
for regs in df['key_regulators']:
    if isinstance(regs, list) and len(regs) > 1:
        # Create edges for all pairs of regulators in same article
        for pair in combinations(regs, 2):
            edges.append(pair)

# Count edge weights
edge_counts = Counter(edges)

# Build graph
G = nx.Graph()
for (reg1, reg2), weight in edge_counts.items():
    if weight >= 2:  # Only include regulators mentioned together 2+ times
        G.add_edge(reg1, reg2, weight=weight)

# Calculate node sizes based on degree centrality
node_sizes = nx.degree_centrality(G)

# Get positions using force-directed layout
pos = nx.spring_layout(G, k=0.5, iterations=50, seed=42)

# Prepare edge traces
edge_trace = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    weight = G[edge[0]][edge[1]]['weight']
    
    edge_trace.append(go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        mode='lines',
        line=dict(width=weight*0.5, color='#4A5568'),  # Lighter gray for dark bg,
        hoverinfo='none',
        showlegend=False
    ))

# Prepare node trace
node_x = []
node_y = []
node_text = []
node_size = []
node_color = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    # Size by centrality
    size = node_sizes[node] * 500 + 20
    node_size.append(size)
    
    # Color by degree (connections)
    degree = G.degree(node)
    node_color.append(degree)
    
    # Hover text
    node_text.append(f"{node}<br>Connections: {degree}<br>Mentions: {sum([1 for regs in df['key_regulators'] if isinstance(regs, list) and node in regs])}")

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers+text',
    text=[node.split()[0] if ' ' in node else node for node in G.nodes()],  # Abbreviated names
    textposition='top center',
    textfont=dict(size=9, color='#FFFFFF'),
    hovertext=node_text,
    hoverinfo='text',
    marker=dict(
        size=node_size,
        color=node_color,
        colorscale='Teal',
        line=dict(width=2, color='white'),
        showscale=True,
        colorbar=dict(
            title='Connections',
            thickness=15,
            len=0.7
        )
    ),
    showlegend=False
)

# Update node marker colors
marker=dict(
    size=node_size,
    color=node_color,
    colorscale='Viridis',  # Changed to lighter scale
    line=dict(width=2, color='white'),
    showscale=True,
    colorbar=dict(
        title='Connections',
        thickness=15,
        len=0.7
    )
),

# For this one specifically, I went with a dark background
fig.update_layout(
    title='Regulator Co-occurrence Network',
    showlegend=False,
    hovermode='closest',
    height=700,
    width=1200,
    font=dict(family='Arial, sans-serif', size=12, color='#FFFFFF'),
    plot_bgcolor='#1A1A2E',  # Dark blue-black
    paper_bgcolor='#1A1A2E',
    title_font=dict(size=18, color='#FFFFFF'),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=20, r=20, t=60, b=20)
)

fig.show()
print(f"\nNetwork Graph complete")
print(f"Nodes (regulators): {len(G.nodes())}")
print(f"Edges (co-occurrences): {len(G.edges())}")


Network Graph complete
Nodes (regulators): 24
Edges (co-occurrences): 20



**Note**: There is clearly a data issue here, because occassionally articles get classified under the wrong market. This is something to be fixed. 

**What I'm Doing:**

- Curating top articles by selecting highest-relevance content from each market
- Deduplicating articles and filtering out low-quality sources
- Presenting structured information including event descriptions, relevance rationale, and source attribution

**What It Shows:**

- The most actionable regulatory developments across all five markets
- Specific regulatory events with clear explanations of institutional investor impact
- Variety of regulatory topics and intensity levels currently affecting each market

In [64]:


# Normalize and deduplicate
df_clean = df.copy()
df_clean['title_normalized'] = df_clean['title'].str.lower().str.replace('[^a-z0-9\s]', '', regex=True)
df_unique = df_clean.drop_duplicates(subset='title_normalized', keep='first')

# Exclude crypto-focused sources
excluded_sources = ['Economictimes.com', 'Bitcoinist', 'Cointelegraph', "newsBTC"]
df_unique = df_unique[~df_unique['source_name'].isin(excluded_sources)]
# Market order
market_order = ['United States', 'United Kingdom', 'Germany', 'Japan', 'France']

# Get top 5 per market
top_articles_list = []
for market in market_order:
    market_top = df_unique[df_unique['market_name'] == market].nlargest(5, 'relevance_score')
    top_articles_list.append(market_top)

top_articles = pd.concat(top_articles_list)

# Prepare table data
table_data = top_articles[['title', 'market_name', 'relevance_score', 'impact_level', 'category', 'what_happened', 'source_name']].copy()

# Color code by market
import matplotlib.colors as mcolors
market_colors = []
for market in table_data['market_name']:
    hex_color = COLORS['markets'][market]
    rgb = mcolors.hex2color(hex_color)
    rgba = f'rgba({int(rgb[0]*255)}, {int(rgb[1]*255)}, {int(rgb[2]*255)}, 0.2)'
    market_colors.append(rgba)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Title</b>', '<b>Market</b>', '<b>Score</b>', '<b>Impact</b>', '<b>Category</b>', '<b>What Happened</b>', '<b>Source</b>'],
        fill_color=COLORS['primary'],
        font=dict(color='white', size=12),
        align='left',
        height=40
    ),
    cells=dict(
        values=[
            table_data['title'],
            table_data['market_name'],
            table_data['relevance_score'],
            table_data['impact_level'],
            table_data['category'].str.replace('_', ' ').str.title(),
            table_data['what_happened'],
            table_data['source_name']
        ],
        fill_color=[market_colors] + [['white']*len(table_data)]*6,
        align='left',
        font=dict(size=11),
        height=30
    )
)])

fig.update_layout(
    title='Most Relevant Regulatory Articles',
    height=900,
    width=1400,
    font=dict(family='Arial, sans-serif'),
    title_font=dict(size=18, color=COLORS['primary']),
    margin=dict(l=20, r=20, t=60, b=20)
)

fig.show()
print(f"\nArticles shown: {len(top_articles)}")
print("\nReview output, then tell me which article titles to exclude")


Articles shown: 25

Review output, then tell me which article titles to exclude


### **Geographic Regulatory Intelligence Map**

**What I'm Doing:**

- Creating an interactive choropleth map of the five markets
- Calculating aggregate metrics for each market including article count, average relevance, and high-impact article count
- Identifying the most frequently mentioned regulator and primary regulatory focus area for each market

**What It Shows:**

- Geographic distribution of regulatory activity with color intensity representing average relevance scores
- Concentration of high-impact regulatory developments across different markets
- Which regulatory authorities are most active in each jurisdiction and the dominant regulatory themes

In [ ]:


# Map country names to ISO codes
country_iso = {
    'United States': 'USA',
    'United Kingdom': 'GBR',
    'Japan': 'JPN',
    'Germany': 'DEU',
    'France': 'FRA'
}

# Prepare market summary data
map_data = []
for market in df['market_name'].unique():
    market_df = df[df['market_name'] == market]
    
    # Get top regulator mentioned
    all_regs = []
    for regs in market_df['key_regulators']:
        if isinstance(regs, list):
            all_regs.extend(regs)
    
    if all_regs:
        top_reg = Counter(all_regs).most_common(1)[0]
        top_regulator = f"{top_reg[0]} ({top_reg[1]} mentions)"
    else:
        top_regulator = "N/A"
    
    # Top category
    top_category = market_df['category'].value_counts().index[0].replace('_', ' ').title()
    
    map_data.append({
        'market': market,
        'iso_code': country_iso[market],
        'article_count': len(market_df),
        'avg_relevance': round(market_df['relevance_score'].mean(), 1),
        'high_impact_count': (market_df['impact_level'] == 'high').sum(),
        'top_regulator': top_regulator,
        'primary_focus': top_category
    })

# Update labels
labels={
    'avg_relevance': 'Avg Relevance',
    'article_count': 'Total Articles',
    'high_impact_count': 'High Impact Articles',
    'top_regulator': 'Most Mentioned Regulator',
    'primary_focus': 'Primary Regulatory Focus'
}

map_df = pd.DataFrame(map_data)

# Create choropleth
fig = px.choropleth(
    map_df,
    locations='iso_code',
    color='avg_relevance',
    hover_name='market',
    hover_data={
        'iso_code': False,
        'article_count': True,
        'avg_relevance': ':.1f',
        'high_impact_count': True,
        'top_regulator': True,
        'primary_focus': True
    },
    color_continuous_scale=[
       [0, '#FFF5E6'],      # Cream
       [0.5, '#FFB84D'],    # Gold
       [1, '#D94A3D']       # Deep red
    ],
    labels={
        'avg_relevance': 'Avg Relevance',
        'article_count': 'Articles',
        'high_impact_count': 'High Impact',
        'top_regulator': 'Top Regulator',
        'primary_focus': 'Policy Focus'
    }
)

fig.update_geos(
    showcountries=True,
    countrycolor='#E0E0E0',
    showcoastlines=True,
    coastlinecolor='#CCCCCC',
    projection_type='natural earth',
    lataxis_range=[20, 70],
    lonaxis_range=[-130, 150],
    bgcolor='#F8F9FA'
)

fig.update_layout(
    title='Regulatory News Intelligence by Market',
    height=300,
    width=1200,
    font=dict(family='Arial, sans-serif', size=12, color='#2C3E50'),
    paper_bgcolor='white',
    title_font=dict(size=18, color=COLORS['primary']),
    margin=dict(l=0, r=0, t=60, b=0),
    coloraxis_colorbar=dict(
        title='Avg<br>Relevance',
        thickness=15,
        len=0.7,
        x=1.02
    )
)

fig.show()
print(f"\nInteractive Map complete - {len(map_df)} markets")

VIZ 6: INTERACTIVE MARKET MAP




Interactive Map complete - 5 markets
